### Load & Inspect Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./Datasets/enhanced_health_insurance_claims.csv")

### Goal:

            Understand data types

            Identify missing values

            Spot obvious issues

In [3]:
df.head()
df.info()
df.describe(include='all')
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ClaimID                  4500 non-null   object 
 1   PatientID                4500 non-null   object 
 2   ProviderID               4500 non-null   object 
 3   ClaimAmount              4500 non-null   float64
 4   ClaimDate                4500 non-null   object 
 5   DiagnosisCode            4500 non-null   object 
 6   ProcedureCode            4500 non-null   object 
 7   PatientAge               4500 non-null   int64  
 8   PatientGender            4500 non-null   object 
 9   ProviderSpecialty        4500 non-null   object 
 10  ClaimStatus              4500 non-null   object 
 11  PatientIncome            4500 non-null   float64
 12  PatientMaritalStatus     4500 non-null   object 
 13  PatientEmploymentStatus  4500 non-null   object 
 14  ProviderLocation        

ClaimID                    0
PatientID                  0
ProviderID                 0
ClaimAmount                0
ClaimDate                  0
DiagnosisCode              0
ProcedureCode              0
PatientAge                 0
PatientGender              0
ProviderSpecialty          0
ClaimStatus                0
PatientIncome              0
PatientMaritalStatus       0
PatientEmploymentStatus    0
ProviderLocation           0
ClaimType                  0
ClaimSubmissionMethod      0
dtype: int64

### Handle Missing Values

In [6]:
# Check Missing

df.isnull().sum().sort_values(ascending=False)

ClaimID                    0
PatientID                  0
ProviderID                 0
ClaimAmount                0
ClaimDate                  0
DiagnosisCode              0
ProcedureCode              0
PatientAge                 0
PatientGender              0
ProviderSpecialty          0
ClaimStatus                0
PatientIncome              0
PatientMaritalStatus       0
PatientEmploymentStatus    0
ProviderLocation           0
ClaimType                  0
ClaimSubmissionMethod      0
dtype: int64

In [ ]:
# Fill numeric with median, categorical with mode

df['ClaimAmount'] = df['ClaimAmount'].fillna(df['ClaimAmount'].median())

# Fill categorical with mode
df['PatientGender'] = df['PatientGender'].fillna(df['PatientGender'].mode()[0])

# Drop if necessary
df = df.dropna(subset=['ClaimDate'])

### Remove Duplicates

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
# if duplicates existed

df = df.drop_duplicates()

In [12]:
# claim ID, patient ID and Provider ID should be unique
df = df.drop_duplicates(subset=['ClaimID'])
df = df.drop_duplicates(subset=['PatientID'])
df = df.drop_duplicates(subset=['ProviderID'])

### Validate & Fix Invalid Entries

In [13]:
# Age Validation

df['PatientAge'].describe()

count    4500.000000
mean       49.838444
std        28.790471
min         0.000000
25%        25.000000
50%        50.500000
75%        75.000000
max        99.000000
Name: PatientAge, dtype: float64

In [14]:
df = df[(df['PatientAge'] >= 0) & (df['PatientAge'] <= 100)]

In [15]:
# Claim amount validation
df = df[df['ClaimAmount'] > 0]


In [16]:
# Standardize Formats

df['ClaimDate'] = pd.to_datetime(df['ClaimDate'], errors='coerce')

In [17]:
# dropping invalid dates

df = df.dropna(subset=['ClaimDate'])

In [18]:
# Standardize Gender

df['PatientGender'].unique()

array(['M', 'F'], dtype=object)

In [ ]:
df['PatientGender'] = df['PatientGender'].str.upper().str.strip()

df['PatientGender'] = df['PatientGender'].replace({
    'MALE': 'M',
    'FEMALE': 'F'
})

### Standardize Codes

In [19]:
df['DiagnosisCode'] = df['DiagnosisCode'].str.upper().str.strip()
df['ProcedureCode'] = df['ProcedureCode'].str.upper().str.strip()

In [20]:
df.head()

,ClaimID,PatientID,ProviderID,ClaimAmount,ClaimDate,DiagnosisCode,ProcedureCode,PatientAge,PatientGender,ProviderSpecialty,ClaimStatus,PatientIncome,PatientMaritalStatus,PatientEmploymentStatus,ProviderLocation,ClaimType,ClaimSubmissionMethod
0,10944daf-f7d5-4e1d-8216-72ffa609fe41,8552381d-7960-4f64-b190-b20b8ada00a1,4a4cb19c-4863-41cf-84b0-c2b21aace988,3807.95,2024-06-07,YY006,HD662,16,M,Cardiology,Pending,90279.43,Married,Retired,Jameshaven,Routine,Paper
1,fcbebb25-fc24-4c0f-a966-749edcf83fb1,327f43ad-e3bd-4473-a9ed-46483a0a156f,422e02dd-c1fd-43dd-8af4-0c3523f997b1,9512.07,2023-05-30,TD052,MH831,27,M,Pediatrics,Approved,130448.02,Single,Student,Beltrantown,Routine,Online
2,9e9983e7-9ea7-45f5-84d8-ce49ccd8a4a1,6f3acdf7-73aa-4afa-9c2e-b25b27bdb5b0,f7733b3f-0980-47b5-a7a0-ee390869355b,7346.74,2022-09-27,ZX832,DG637,40,F,Cardiology,Pending,82417.54,Divorced,Employed,West Charlesport,Emergency,Online
3,a06273ed-44bb-452b-bbad-8618de080494,5d58e183-701e-406c-a8c6-5b73cac5e912,f7a04581-de96-44ee-b773-8adac02baa59,6026.72,2023-06-25,KR421,KG326,65,M,Neurology,Pending,68516.96,Widowed,Student,West Aprilhaven,Routine,Phone
4,f702a717-254b-4cff-a0c7-8395db2f6616,8a8ebdf6-3af0-4f14-82f3-37b937c3d270,b80b9e77-97f0-47d7-b561-19f9658a7bdf,1644.58,2023-07-24,LZ261,CX805,24,M,General Practice,Pending,84122.17,Married,Student,Lake Michele,Inpatient,Phone


### Create Member-Month Metrics

In [21]:
df['YearMonth'] = df['ClaimDate'].dt.to_period('M')   # Extract year month

### Count Claims per Member per Month

In [30]:
member_month = df.groupby(['PatientID', 'YearMonth']).size().reset_index(name='ClaimCount')
member_month

,PatientID,YearMonth,ClaimCount
0,000f4454-0d12-4a00-8cbd-943e3730f99b,2022-09,1
1,001ed9ce-670b-478d-80c7-44b3fe8a8ef6,2022-10,1
2,00236546-ccac-4607-aef0-f22518e0e459,2023-05,1
3,002e8db4-4731-48c5-8b65-241f0f20b726,2024-06,1
4,003662ef-f5b8-4920-8873-45ea0d926b0c,2022-08,1
...,...,...,...
4495,ffcd16ec-7730-4411-9efb-dc8d2f61ae20,2023-08,1
4496,ffdea2ae-1a13-44b2-878d-805682feda8b,2023-03,1
4497,ffe50db6-6ccd-4b2c-a43d-d01d8e232f66,2023-09,1
4498,fff3236c-6945-4e8a-8e1f-ebb00610d188,2024-02,1


### Final Validation Checklist

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ClaimID                  4500 non-null   object        
 1   PatientID                4500 non-null   object        
 2   ProviderID               4500 non-null   object        
 3   ClaimAmount              4500 non-null   float64       
 4   ClaimDate                4500 non-null   datetime64[ns]
 5   DiagnosisCode            4500 non-null   object        
 6   ProcedureCode            4500 non-null   object        
 7   PatientAge               4500 non-null   int64         
 8   PatientGender            4500 non-null   object        
 9   ProviderSpecialty        4500 non-null   object        
 10  ClaimStatus              4500 non-null   object        
 11  PatientIncome            4500 non-null   float64       
 12  PatientMaritalStatus     4500 non-

In [23]:
df.describe()

,ClaimAmount,ClaimDate,PatientAge,PatientIncome
count,4500.000000,4500,4500.000000,4500.000000
mean,5014.203867,2023-07-05 06:48:38.400000,49.838444,84384.284084
min,100.120000,2022-07-09 00:00:00,0.000000,20006.870000
25%,2509.072500,2023-01-07 18:00:00,25.000000,52791.905000
50%,5053.765000,2023-07-03 00:00:00,50.500000,84061.205000
75%,7462.452500,2023-12-29 00:00:00,75.000000,115768.417500
max,9997.200000,2024-07-08 00:00:00,99.000000,149957.520000
std,2866.291066,NaN,28.790471,37085.908878


In [24]:
df.isnull().sum()

ClaimID                    0
PatientID                  0
ProviderID                 0
ClaimAmount                0
ClaimDate                  0
DiagnosisCode              0
ProcedureCode              0
PatientAge                 0
PatientGender              0
ProviderSpecialty          0
ClaimStatus                0
PatientIncome              0
PatientMaritalStatus       0
PatientEmploymentStatus    0
ProviderLocation           0
ClaimType                  0
ClaimSubmissionMethod      0
YearMonth                  0
dtype: int64

In [25]:
df.duplicated().sum()

np.int64(0)

In [26]:
df.head()


,ClaimID,PatientID,ProviderID,ClaimAmount,ClaimDate,DiagnosisCode,ProcedureCode,PatientAge,PatientGender,ProviderSpecialty,ClaimStatus,PatientIncome,PatientMaritalStatus,PatientEmploymentStatus,ProviderLocation,ClaimType,ClaimSubmissionMethod,YearMonth
0,10944daf-f7d5-4e1d-8216-72ffa609fe41,8552381d-7960-4f64-b190-b20b8ada00a1,4a4cb19c-4863-41cf-84b0-c2b21aace988,3807.95,2024-06-07,YY006,HD662,16,M,Cardiology,Pending,90279.43,Married,Retired,Jameshaven,Routine,Paper,2024-06
1,fcbebb25-fc24-4c0f-a966-749edcf83fb1,327f43ad-e3bd-4473-a9ed-46483a0a156f,422e02dd-c1fd-43dd-8af4-0c3523f997b1,9512.07,2023-05-30,TD052,MH831,27,M,Pediatrics,Approved,130448.02,Single,Student,Beltrantown,Routine,Online,2023-05
2,9e9983e7-9ea7-45f5-84d8-ce49ccd8a4a1,6f3acdf7-73aa-4afa-9c2e-b25b27bdb5b0,f7733b3f-0980-47b5-a7a0-ee390869355b,7346.74,2022-09-27,ZX832,DG637,40,F,Cardiology,Pending,82417.54,Divorced,Employed,West Charlesport,Emergency,Online,2022-09
3,a06273ed-44bb-452b-bbad-8618de080494,5d58e183-701e-406c-a8c6-5b73cac5e912,f7a04581-de96-44ee-b773-8adac02baa59,6026.72,2023-06-25,KR421,KG326,65,M,Neurology,Pending,68516.96,Widowed,Student,West Aprilhaven,Routine,Phone,2023-06
4,f702a717-254b-4cff-a0c7-8395db2f6616,8a8ebdf6-3af0-4f14-82f3-37b937c3d270,b80b9e77-97f0-47d7-b561-19f9658a7bdf,1644.58,2023-07-24,LZ261,CX805,24,M,General Practice,Pending,84122.17,Married,Student,Lake Michele,Inpatient,Phone,2023-07


In [28]:
df.to_csv("./Datasets/cleaned_healthcare_claims.csv", index=False)